In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy
import scipy
import math
import matplotlib as mpl
import matplotlib.dates as mpd
import pylab as plt
import datetime as dtm
import sys
import os
#import socket
#import lmod
# lmod.load('system')
# lmod.load('texlive')
# lmod.
#
# TODO: phase out unreferenced hpc_lib calls...
import hpc_lib

/home/users/myoder96/Codes/HPC_analytics/hpc_lib.py:2624: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def process_sacct_row(rw, delim='\t', headers=None, types_dict={}, RH={}):
/home/users/myoder96/Codes/HPC_analytics/hpc_lib.py:2639: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_modulus_stats(X,Y, a_mult=1., a_mod=1., qs=numpy.array([.5, .75, .95])):


In [3]:
n_cpus = 4
#print('** epoch: {}'.format(mpd.get_epoch()))
if 'SLURM_CPUS_PER_TASK' in os.environ.keys():
    n_cpus = int(os.environ['SLURM_CPUS_PER_TASK'])
#
print(f'*** n_cpus: {n_cpus}')


*** n_cpus: 6


In [6]:
SHP = hpc_lib.SH_PART_obj()

/home/users/myoder96/Codes/HPC_analytics/hpc_lib.py:2624: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def process_sacct_row(rw, delim='\t', headers=None, types_dict={}, RH={}):
/home/users/myoder96/Codes/HPC_analytics/hpc_lib.py:2639: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_modulus_stats(X,Y, a_mult=1., a_mod=1., qs=numpy.array([.5, .75, .95])):


In [7]:
print('** ', SHP.SP)

**  [' partition            nodes          CPU cores             GPUs                  job runtime      mem/core         per-node', ' name         public    idle  total    idle  total queued    idle  total queued  default maximum  default maximum     cores   mem(GB)  gpus', '----------------------------------------------------------------------------------------------------------------------------------------------------', ' normal*      yes          0    179     238   4820   6828       0      0      0       2h      7d      6GB     8GB     20-32   128-256     0', ' bigmem       yes          0      9     260    504      5       0      0      0       2h      1d      8GB    64GB    24-128  384-4096     0', ' gpu          yes          0     26     191    748    317      21    104    110       1h      2d      7GB    12GB     20-32   191-256     4', ' dev          yes          0      4      44    104      2      62     64      0       1h      2h      6GB     8GB     20-32   128-256  0-32', '

In [9]:
for rw in SHP.SP:
    print('** ', rw.split())

**  ['partition', 'nodes', 'CPU', 'cores', 'GPUs', 'job', 'runtime', 'mem/core', 'per-node']
**  ['name', 'public', 'idle', 'total', 'idle', 'total', 'queued', 'idle', 'total', 'queued', 'default', 'maximum', 'default', 'maximum', 'cores', 'mem(GB)', 'gpus']
**  ['----------------------------------------------------------------------------------------------------------------------------------------------------']
**  ['normal*', 'yes', '0', '179', '238', '4820', '6828', '0', '0', '0', '2h', '7d', '6GB', '8GB', '20-32', '128-256', '0']
**  ['bigmem', 'yes', '0', '9', '260', '504', '5', '0', '0', '0', '2h', '1d', '8GB', '64GB', '24-128', '384-4096', '0']
**  ['gpu', 'yes', '0', '26', '191', '748', '317', '21', '104', '110', '1h', '2d', '7GB', '12GB', '20-32', '191-256', '4']
**  ['dev', 'yes', '0', '4', '44', '104', '2', '62', '64', '0', '1h', '2h', '6GB', '8GB', '20-32', '128-256', '0-32']
**  ['---------------------------------------------------------------------------------------------